In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from functions import *

In [2]:
#import plotly
#import plotly.express as px
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#import plotly.graph_objects as go
#init_notebook_mode(connected=True)

Data Source 'fets': https://datos.gob.es/es/catalogo/a09002970-delitos-y-hechos-delicuenciales-producidos-en-la-red-viaria-patrimonio

In [4]:
fets = pd.read_csv('data/fets.csv')
patrulles = pd.read_csv('data/mitjana_patrulles.csv')

##### The following list/index summarizes the actions I am taking in order to clean the data. So it is completely ready to create a visualization using Tableau or any other BI tool.
- #### Data Cleaning
    - *1.1 Standarize col names and translate to English*
    - *1.2 Standarize upper/lower case for RP and ABT values*
    - *1.3 Remove RP and ABP in front of the value name (area)*
    - *1.4 Create a new col called 'Date':* This column will be datetime type and will include the month and the year when the offense happened.
    - *1.5 Unify value names for the 'offenses_crimes' column.*


In [5]:
# 1.1 Standarize col names to make it easier
fets = fets.rename(columns = {'Mes':'month', 'Nom mes':'month_name', 'Any':'year', 'Regió Policial (RP)':'RP', 'Àrea Regional de Trànsit (ART)/ Àrea Bàsica Policial (ABP)':'ABP', 'Tipus de fet':'offense_crime', 'Nombre':'number'})
patrulles = patrulles.rename(columns = {'Mes':'month', 'Nom mes':'month_name', 'Any':'year', 'Servei origen dotació':'place', 'Mitjana patrulles diàries':'daily_patrols_mean'})
display(fets.head())
display(patrulles.head())

,month,month_name,year,RP,ABP,offense_crime,number
0,1,Gener,2011,RP CENTRAL,ABP OSONA,Conduir sota els efectes d'alcohol,11
1,1,Gener,2011,RP CENTRAL,ABP OSONA,Negativa a sotmetre's a les proves,4
2,1,Gener,2011,RP CENTRAL,ABP OSONA,Conduir sense permís per pèrdua de vig.\nper p...,6
3,1,Gener,2011,RP CENTRAL,ABP OSONA,Conduir sense haver obtingut mai permís o\nlli...,2
4,1,Gener,2011,RP CENTRAL,ABP OSONA,Cond. amb priv. judicial caut. o def.\npermís/...,1


,month,month_name,year,place,daily_patrols_mean
0,1,Gener,2011,Serveis Centrals,176
1,1,Gener,2011,Serveis Regionals - RP Central,34
2,1,Gener,2011,ABP Osona,22
3,1,Gener,2011,ABP Anoia,19
4,1,Gener,2011,ABP Bages,27


In [6]:
# 1.2 Standarize upper/lower case for RP and ABT values
# the reason why it looks like this values are repeat is because
# from 2011 until 2017 they used upper case
# from 1017 until 2021 they user lower case
fets['RP'] = fets['RP'].str.upper()
fets['ABP'] = fets['ABP'].str.upper()

In [7]:
# 1.3 Remove RP and ABP in from of the value name (area)
fets['ABP'] = fets['ABP'].str.replace('ABP',' ')
fets['RP'] = fets['RP'].str.replace('RP',' ')

In [8]:
# 1.4 Creating a new col called 'Date'. This column will be .datetime type and will include the month and the year when the offense happened
fets['date'] = pd.to_datetime(fets[['year', 'month']].assign(DAY=1))

In [9]:
# 1.5 Let's now unify some value names that have been written in different ways but mean the same.
# Let's keep the count of our unique values for that col to see the amount of unique values we cold describe better.

fets['offense_crime'].nunique()

In [11]:
# Thefts inside vehicles are divided by 'theft' or 'petty theft'
# in this case we don't need to know the severity of the offence, just the type
# so I will make them all (petty or note) count as 'theft' 'Furt'
fets.loc[fets['offense_crime'].str.contains('lleu'), 'offense_crime'] = 'Furt'
fets.loc[fets['offense_crime'].str.contains('Falta de furt'), 'offense_crime'] = 'Furt'

In [12]:
# Damage has also been recorded as ´Damage´ and ´Damage fault´ so will put them all together as ´Damage´ ´Danys´
# We will also include 'Slight Damage' as 'Damage' 'Danys'
fets.loc[fets['offense_crime'].str.contains('Falta de danys'), 'offense_crime'] = 'Danys'
fets.loc[fets['offense_crime'].str.contains('Danys (lleu)'), 'offense_crime'] = 'Danys'

/var/folders/6m/xnrf68g93dj9cnj3lwnc18b00000gn/T/ipykernel_70783/1174277919.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  fets.loc[fets['offense_crime'].str.contains('Danys (lleu)'), 'offense_crime'] = 'Danys'


In [13]:
# Driving with precautionary or definitive judicial deprivation, without a driving license.
# This offense has ben recorded in two different ways but both meaning the same.
# Let's put them together under the value 'Conduir amb privació judicial cautelar o definitiva de permís/llicència'
fets.loc[fets['offense_crime'].str.contains('Cond. amb priv. judicial caut. o def.'), 'offense_crime'] = 'Conduir amb privació judicial cautelar o definitiva de permís/llicència'

In [14]:
# Driving without a license due to loss of points
# This offense has been recorded in two different ways but both meaning the same.
# Let's put them together under the value 'Conduir sense permís per pèrdua de vigència per punts'

fets.loc[fets['offense_crime'].str.contains('Conduir sense permís per pèrdua de vig.'), 'offense_crime'] = 'Conduir sense permís per pèrdua de vigència per punts'

In [15]:
# Bank scams are recorded as 'bank scams' and 'banks scams (credit card)' 
# I will put them all together as 'bank scams' 'falta d'estafa (bancaria)'
fets.loc[fets['offense_crime'].str.contains('targetes'), 'offense_crime'] = "Falta d'estafa (bancària)"

In [20]:
fets.loc[fets['offense_crime'].str.contains('Robatori amb força'), 'offense_crime'] = 'Robatori de vehicle amb força/violència/intimidació'
fets.loc[fets['offense_crime'].str.contains("Robatori o furt d'ús de vehicle"), 'offense_crime'] = 'Robatori de vehicle amb força/violència/intimidació'
fets.loc[fets['offense_crime'].str.contains('Robatori amb violència i/o intimidació'), 'offense_crime'] = 'Robatori de vehicle amb força/violència/intimidació'


In [24]:
# It seems like we could clean our data in a quite efficient way :)
fets['offense_crime'].nunique()

191

In [25]:
# Saving data for Tableau
fets.to_csv('data/tableau_fets.csv')
#patrulles.to_csv('data/tableau_patrulles.csv')